In [113]:
import pandas as pd
import json
import numpy
import geopy.distance
import plotly.express as px
import math
import sys
from collections import Counter

In [44]:
#dataset = pd.read_csv("partial_added_points.csv")
dataset = pd.read_csv("partial_subtracks_with_extra_points_after_dividing_only_city_centre.csv")

In [102]:
def point_distance(point1, point2):
    p1 = (point1[0], point1[1])
    p2 = (point2[0], point2[1])
    dist = geopy.distance.distance(p1, p2).m
    if dist < 30:
        return 0
    else:
        return 1

In [103]:
numpy.set_printoptions(threshold=sys.maxsize)


def create_matrix(track1, track2):
    matrix = numpy.zeros((len(track1) + 1, len(track2) + 1))

    matrix[0, 0] = 0

    for r in range(1, len(track1) + 1):
        matrix[r, 0] = numpy.inf

    for c in range(1, len(track2) + 1):
        matrix[0, c] = numpy.inf

    for i in range(1, len(track1) + 1):
        for j in range(1, len(track2) + 1):
            dist = point_distance(track1[i - 1], track2[j - 1])
            matrix[i, j] = (
                min(matrix[i - 1, j], min(matrix[i, j - 1], matrix[i - 1, j - 1]))
                + dist
            )

    return matrix


track1 = dataset.loc[dataset["id"] == "T2"].values.tolist()
track2 = dataset.loc[dataset["id"] == "T18"].values.tolist()
# track1 = [[1.0,1.2], [1.3, 1.7], [1.3,1.9], [1.5,1.6]]
# track2 = [[1.1,1.2], [1.4,1.4], [1.3,1.9], [1.4,1.7], [1.5,1.6]]
matrix = create_matrix(track1, track2)
print(matrix)

[[  0.  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
   inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  i

In [117]:
def backtrack(matrix, track1, track2):
    a = []
    i = len(track1)
    j = len(track2)

    while i > 0 and j > 0:

        minimum = min(matrix[i - 1, j - 1], min(matrix[i - 1, j], matrix[i, j - 1]))
        if matrix[i - 1, j] == minimum:
            i = i - 1
        elif matrix[i, j - 1] == minimum:
            j = j - 1
        elif matrix[i - 1, j - 1] == minimum:
            j = j - 1
            i = i - 1
        else:
            print("hallais")
        if (
            point_distance(
                (track1[i - 1][0], track1[i - 1][1]),
                (track2[j - 1][0], track2[j - 1][1]),
            )
            == 0
        ):
            point = [track1[i - 1][0], track1[i - 1][1], track2[0][2]]
            if len(a) == 0 or a[0] != point:
                a.insert(0, point)
    return a


align = backtrack(matrix, track1, track2)
print(align)

[[41.146191, -8.610443999999998, 'T18'], [41.146191, -8.610444, 'T18'], [41.1462855, -8.610277499999999, 'T18'], [41.14638, -8.610111, 'T18'], [41.1464745, -8.609944500000001, 'T18'], [41.14656900000001, -8.609778000000002, 'T18'], [41.14666350000001, -8.609611500000003, 'T18'], [41.14675800000001, -8.609445000000004, 'T18'], [41.146758, -8.609445000000001, 'T18'], [41.146878, -8.609262000000001, 'T18'], [41.146998, -8.609079000000001, 'T18'], [41.147118000000006, -8.608896000000001, 'T18'], [41.147118, -8.608896, 'T18'], [41.147127, -8.608968, 'T18'], [41.1473295, -8.6088375, 'T18']]


In [47]:
track_ids = pd.read_csv("partial_subtracks_ids_add_points_after_dividing.csv").values.tolist()


In [119]:
def Dynamic_Time_Warping(dataset, trackid, nr):
    track = dataset.loc[dataset["id"] == trackid]
    # track = pd.DataFrame.drop_duplicates(fulltrack)
    print("track1: ", trackid)
    alignments = []
    for j in range(nr + 1, len(track_ids)):
        next_track = dataset.loc[dataset["id"] == track_ids[j][0]]
        matrix = create_matrix(track.values.tolist(), next_track.values.tolist())
        alignment = backtrack(matrix, track.values.tolist(), next_track.values.tolist())
        if len(alignment) > 3:
            alignments.extend(alignment)
    return alignments


align = Dynamic_Time_Warping(dataset, "T2", 1)
print(align)

track1:  T2
[[41.146191, -8.610443999999998, 'T18'], [41.146191, -8.610444, 'T18'], [41.1462855, -8.610277499999999, 'T18'], [41.14638, -8.610111, 'T18'], [41.1464745, -8.609944500000001, 'T18'], [41.14656900000001, -8.609778000000002, 'T18'], [41.14666350000001, -8.609611500000003, 'T18'], [41.14675800000001, -8.609445000000004, 'T18'], [41.146758, -8.609445000000001, 'T18'], [41.146878, -8.609262000000001, 'T18'], [41.146998, -8.609079000000001, 'T18'], [41.147118000000006, -8.608896000000001, 'T18'], [41.147118, -8.608896, 'T18'], [41.147127, -8.608968, 'T18'], [41.1473295, -8.6088375, 'T18'], [41.14557, -8.610876000000001, 'T32'], [41.145579000000005, -8.610858, 'T32'], [41.145768, -8.610903, 'T32'], [41.145909, -8.61075, 'T32'], [41.14605, -8.610596999999999, 'T32'], [41.146191, -8.610443999999998, 'T32'], [41.146191, -8.610444, 'T32'], [41.1462855, -8.610277499999999, 'T32'], [41.14638, -8.610111, 'T32'], [41.1464745, -8.609944500000001, 'T32'], [41.14656900000001, -8.60977800000

In [54]:
def create_list_of_points(tracknr, alignments):

    track = dataset.loc[dataset["id"] == tracknr]
    track = pd.DataFrame.drop_duplicates(track)
    points = pd.DataFrame(columns=["lat", "lon", "ids"])
    align = pd.DataFrame(alignments, columns=["lat", "lon", "id"])

    for i in range(len(track)):
        lat = align.loc[align["lat"] == track.iloc[i]["lat"]]
        points.loc[i] = [
            track.iloc[i]["lat"],
            track.iloc[i]["lon"],
            lat["id"].values.tolist(),
        ]
    return points

In [55]:
def remove_infrequent_points(points):
    point_list = pd.DataFrame(points, columns=["lat", "lon", "ids"])
    for i in range(len(point_list)):
        val = len(point_list.loc[i]["ids"])
        point_list.at[i, "ids"] = val
    return point_list

In [56]:
def gather_sequences(points):
    i = 0
    final_records = pd.DataFrame(columns=["polyline", "value"])
    for group in range(points.iloc[0]["group"], -1, -1):
        records = pd.DataFrame(columns=["polyline", "value"])
        sequence = []
        value = 0
        while i < len(points) and points.iloc[i]["group"] == group:
            if value == points.iloc[i]["ids"] or value == 0:
                sequence.append(
                    [
                        points.index[i],
                        points.iloc[i]["lat"],
                        points.iloc[i]["lon"],
                    ]
                )
                value = points.iloc[i]["ids"]
                i += 1
            else:
                sequence.sort()
                new_sequence = sequence.copy()
                records.loc[len(records)] = [
                    new_sequence,
                    (len(new_sequence) * 1.0) + value,
                ]
                value = points.iloc[i]["ids"]

        sequence.sort()
        new_sequence = sequence.copy()
        records.loc[len(records)] = [
            new_sequence,
            (len(new_sequence) * 1.0) + value,
        ]
        if len(records) < 2:
            final_records = pd.concat([final_records, records])
        else:
            highest_value = records["value"].idxmax()
            final_records = final_records.append(records.loc[highest_value])

    return final_records

In [120]:
def hele_sullamitten(dataset):
    most_tracked_roads = pd.DataFrame(columns=["polyline", "value"])
    for i in range(len(track_ids)):
        track = track_ids[i][0]
        alignment = Dynamic_Time_Warping(dataset, track, i)
        points = create_list_of_points(track, alignment)
        new_points = remove_infrequent_points(points)
        new_points = new_points[new_points.ids > 8]
        if not new_points.empty:
            new_points["group"] = (new_points.index.to_series().diff() > 1).cumsum()
            new_points.sort_values(by=["group", "ids"], ascending=False, inplace=True)
            records = gather_sequences(new_points)
            most_tracked_roads = pd.concat([most_tracked_roads, records])
            print(most_tracked_roads)
    return most_tracked_roads


res = hele_sullamitten(dataset)

track1:  T1
                                            polyline  value
2  [[2, 41.148855000000005, -8.585685000000002], ...   28.0
track1:  T2
                                            polyline  value
2  [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
track1:  T2334
                                            polyline  value
2  [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0  [[9, 41.159943000000005, -8.604513000000006], ...   12.0
track1:  T3
                                            polyline  value
2  [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0  [[9, 41.159943000000005, -8.604513000000006], 

track1:  T15
                                            polyline  value
2  [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0  [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0  [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0  [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1  [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0  [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0  [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0  [[4, 41.153832, -8.582021999999998], [5, 41.15...   16.0
0  [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0  [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0               [[4, 41.144724, -8.606708999999999]]   16.0
0                        [[1, 41.144724, -8.606673]]   16.0
0  [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0  [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0  [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0  [[69, 41.168277, -8.5876

                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   16.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0                [[4, 41.144724, -8.606708999999999]]   16.0
0                         [[1, 41.144724, -8.606673]]   16.0
0   [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0   [[69, 41.168277, -8.

                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   16.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0                [[4, 41.144724, -8.606708999999999]]   16.0
0                         [[1, 41.144724, -8.606673]]   16.0
0   [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0   [[69, 41.168277, -8.

                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
0   [[50, 41.160816, -8.581499999999993], [51, 41....   14.0
0   [[22, 41.156487, -8.582985000000004], [23, 41....   14.0
0   [[4, 41.153832, -8.582021999999998], [5, 41.15...   16.0
0   [[1, 41.1489, -8.585640000000001], [2, 41.1489...   22.0
0   [[3, 41.152923, -8.581185000000001], [4, 41.15...   14.0
0                [[4, 41.144724, -8.606708999999999]]   16.0
0                         [[1, 41.144724, -8.606673]]   16.0
0   [[1, 41.148603, -8.585703], [2, 41.148603, -8....   51.0
0   [[60, 41.160321, -8.581725000000004], [61, 41....   12.0
0   [[36, 41.15640600000001, -8.582985000000003], ...   12.0
0   [[69, 41.168277, -8.

...

track1:  T179814407
track1:  T179174411
track1:  T17984415
track1:  T17904427
track1:  T18014434
                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0   [[9, 41.15254949999999, -8.6099265], [10, 41.1...   14.0
0   [[4, 41.151672000000005, -8.609634000000002], ...   11.0
0   [[0, 41.151222, -8.609724], [1, 41.151222, -8....   18.0
0   [[11, 41.147505, -8.621433], [12, 41.147505, -...   54.0
0   [[27, 41.147568, -8.609035500000001], [28, 41....   21.0

[256 rows x 2 columns]
track1:  T181
track1:  T181654460
track1:  T181664465
track1:  T181674471
                                             polyline  value
2   [[2, 41.

track1:  T197265055
track1:  T197265069
track1:  T198
track1:  T198375084
track1:  T198855088
track1:  T199155103
track1:  T200
track1:  T201
track1:  T201215170
track1:  T202
                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0   [[9, 41.161257, -8.608049999999999], [10, 41.1...   23.0
0   [[51, 41.14647900000001, -8.609723999999998], ...   12.0
0   [[44, 41.146083, -8.610912000000003], [45, 41....   18.0
2   [[34, 41.145939, -8.612910000000001], [35, 41....   18.0
0                        [[26, 41.145939, -8.614332]]   14.0

[292 rows x 2 columns]
track1:  T2031115234
track1:  T204465238
track1:  T205155246
track1:

track1:  T224
track1:  T224966165
track1:  T224506174
track1:  T224266177
track1:  T2241096194
track1:  T224366198
track1:  T2241166210
track1:  T2241166218
track1:  T224256290
track1:  T224266300
track1:  T224176322
track1:  T224156325
track1:  T225976345
track1:  T225466351
track1:  T225476367
track1:  T225486371
track1:  T225996403
track1:  T225236406
track1:  T225406416
track1:  T226
track1:  T226796448
track1:  T227
track1:  T227196462
track1:  T22716476
track1:  T22846504
track1:  T22846510
track1:  T2281006514
track1:  T22876518
track1:  T228156528
track1:  T228156572
track1:  T230
                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                    

track1:  T247
track1:  T247797107
track1:  T248
                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0   [[4, 41.153688, -8.613387000000003], [5, 41.15...   28.0
0   [[129, 41.166009, -8.606448000000011], [130, 4...   12.0
5   [[11, 41.147991000000005, -8.622602999999998],...   54.0
4   [[58, 41.14891636363636, -8.624960999999997], ...   64.0
4   [[29, 41.147778, -8.620254], [30, 41.147716, -...   37.0

[353 rows x 2 columns]
track1:  T249
                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0

track1:  T266
track1:  T267
track1:  T2671037532
track1:  T268
                                             polyline  value
2   [[2, 41.148855000000005, -8.585685000000002], ...   28.0
0   [[35, 41.149899000000005, -8.607348], [36, 41....   24.0
0   [[5, 41.146191, -8.610443999999998], [6, 41.14...   32.0
0   [[9, 41.159943000000005, -8.604513000000006], ...   12.0
1   [[1, 41.148828, -8.585730000000002], [2, 41.14...   25.0
..                                                ...    ...
0   [[71, 41.148513, -8.624907000000002], [72, 41....   18.0
0   [[50, 41.14761940384614, -8.621412057692318], ...   14.0
0   [[0, 41.148027, -8.612406000000002], [1, 41.14...   13.0
0   [[100, 41.152545, -8.629271999999998], [101, 4...   14.0
0   [[79, 41.151915, -8.625888000000007], [80, 41....   16.0

[383 rows x 2 columns]
track1:  T269
track1:  T269107565
track1:  T270957576
track1:  T270597581
track1:  T270517592
track1:  T270357597
track1:  T271
track1:  T273
                                       

In [121]:
pd.DataFrame.to_csv(res, "./DTW_result_1.csv", index=True)